# Plug-in classifier

Suppose we are given a set of classes $\mathcal{Y} = \{0,1,...,K\}$, a dataset $X = (x_1,...,x_n)$ and a set of ouputs $(y_1,...,y_n)$ where $x_i \in \mathbb{R}^d$ and $y_i \in \mathcal{Y}$. If we are given $P(Y=y)$(class prior) and $P(X=x|Y=y)$(data likelihood given class), then the Bayes classifier is given by

$$f(x) = \arg \max_{y \in \mathcal{Y}} P(X=x | Y=y)P(Y=y).$$

However, typically we do not know what the class prior and data likelihood are. In this case one can approximate $P(X=x|Y=y)$, $P(Y=y)$. One possible approach is to estimate the Bayes classifier via maximum likelihood estimate(MLE). The procedure is as follows:

1. One can check that the MLE of $P(y)$ is
$$ \hat{\pi}_y = \frac{1}{n} \sum_{i=1}^n \mathbb{1}(y_i = y).$$

2. Given $y$, assume that the data is generated from a gaussian distribution with density $N(x|\mu_y, \Sigma_y)$. Let
$$ n_y = \sum_{i=1}^n \mathbb{1}(y_i=y).$$
Then the MLE estimate of $(\mu_y, \Sigma_y)$ is:
$$ \hat{\mu}_y = \frac{1}{n_y} \sum_{i=1}^n \mathbb{1}(y_i = y) x_i,$$
$$ \hat{\Sigma}_y = \frac{1}{n_y} \sum_{i=1}^n \mathbb{1}(y_i = y) (x_i - \hat{\mu}_y)(x_i - \hat{\mu}_y)^T.$$

3. The plug-in classifier is given by

$$\hat{f}(x) = \arg \max_{y \in \mathcal{Y}} \hat{\pi}_y |\hat{\Sigma}_y|^{-1/2} 
\exp\left( -\frac{1}{2} (x-\hat{\mu}_y)^T \hat{\Sigma}_y^{-1} (x - \hat{\mu}_y) \right).$$

## Application: Iris dataset

In the following we implement and apply the plug-in classifier to Iris dataset that can be found at: https://archive.ics.uci.edu/ml/datasets/iris. 

Import, load and preprocess data:

In [1]:
from __future__ import division
import numpy as np
import pandas as pd

## data preprocessing
data = pd.read_csv('iris.csv', sep=",", header=None)

for n in range(data.shape[0]):
    if data.get_value(n, 4) == 'Iris-setosa':
        data.set_value(n, 4, 0)
    if data.get_value(n, 4) == 'Iris-versicolor':
        data.set_value(n, 4, 1)
    if data.get_value(n, 4) == 'Iris-virginica':
        data.set_value(n, 4, 2)

data = data.as_matrix()
np.random.shuffle(data)
print(data[1:10])
print(data.shape)

[[4.5 2.3 1.3 0.3 0]
 [6.6 3.0 4.4 1.4 1]
 [6.0 3.4 4.5 1.6 1]
 [6.8 3.0 5.5 2.1 2]
 [6.3 2.8 5.1 1.5 2]
 [6.1 3.0 4.6 1.4 1]
 [4.9 2.4 3.3 1.0 1]
 [6.3 3.4 5.6 2.4 2]
 [4.4 2.9 1.4 0.2 0]]
(150, 5)


Each row corresponds to one iris (flower), the first four columns are certains parameters of the flower, for details see the link above. The last column is the type of the iris $\in \{0, 1, 2\}$.

Here is the implementation of the plug-in classifier we discussed above:

In [2]:
def pluginClassifier(X_train, y_train, X_test, classes=3):
    rows_test = X_test.shape[0]
    rows = X_train.shape[0]
    columns = X_train.shape[1] # aka number of classes
    proby = np.zeros((rows_test, classes)) # probabilities
    piy= np.zeros(classes) # mle for prior
    ny = np.zeros(classes) # number of elements in class
    muy = np.zeros((classes, columns)) # mle for average
    sigmay = np.zeros((columns, columns, classes)) # mle for covariance
    invy = np.zeros((columns, columns, classes)) # inverses of sigmay
    sqrty = np.zeros(classes) # sqrt of determinant of invy

    ## compute mle(maximum likelihood estimate) for prior and average
    for n in range(classes):
        sumx = np.zeros(X_train.shape[1])
        for k in range(rows):
            if y_train[k] == n: # or y_train[k] == n+1 depending on indexing
                ny[n] += 1
                sumx = sumx + X_train[k, :]
        piy[n] = ny[n]/rows
        muy[n, :] = (1/ny[n])*sumx


    ## compute mle for covariance
    for n in range(classes):
        for k in range(rows):
            if y_train[k] == n:
                aux = X_train[k, :] - muy[n]
                sigmay[:, :, n] = sigmay[:, :, n] + np.outer(aux, aux)
        sigmay[:, :, n] = (1/ny[n])*sigmay[:, :, n]
        invy[:, :, n] = np.linalg.inv(sigmay[:, :, n])
        sqrty[n] = np.sqrt(np.absolute(np.linalg.det(invy[:, :, n])))

    ## plug-in classifier: computing not rescale p(x, y)
    for k in range(rows_test):
        for n in range(classes):
            vec = X_test[k, :] - muy[n, :]
            aux = (-0.5)*np.dot(vec.T, np.dot(invy[:, :, n], vec))
            proby[k, n] = piy[n]*sqrty[n]*np.exp(aux)

    aux = np.sum(proby, axis=1)

    ## rescaling probabilities p(x,y)
    for k in range(rows_test):
        for n in range(classes):
            proby[k, n] = proby[k, n]/aux[k]

    return proby

The function return a matrix of (normalized) probabilities, where proby(i,k) is the probability that that the i-th element of the test set is in class k. Knowing these probabilities we can predict the classes. 

Since we are not tweaking any hyperparameters here, I tested the performance of the algorithm shuffling the dataset many times, splitting it into train and test set, checking the test accuracy and finally averaging over the number of iterations.

In [3]:
## classifying according to output probabilities several times for shuffled data
test_acc = 0
iterations = 100
for l in range(iterations):
    # print(l+1)
    np.random.shuffle(data)
    X_train = data[0:120, 0:4]
    y_train = data[0:120, 4]
    X_test = data[120:150, 0:4]
    y_test = data[120:150, 4]
    probabilities = pluginClassifier(X_train, y_train, X_test)
    rows_test = X_test.shape[0]
    y_out = np.zeros(rows_test)
    ## classifying according to the highest probability
    for k in range(rows_test):
        vec = probabilities[k, :]
        y_out[k] = np.argmax(vec)
        test_acc += np.sum(y_out[k]==y_test[k] )/30

print("Average accuracy: ", test_acc/iterations)

Average accuracy:  0.974666666667
